# Scraping and Storing

[David J. Thomas](mailto:dave.a.base@gmail.com), [thePort.us](http://thePort.us)<br />
Instructor of Ancient History and Digital Humanities,<br />
Department of History,<br />
[University of South Florida](https://github.com/usf-portal)

---

## This workbook will...

* Create a local db to store the data
* Scrape/save charter info from ASC and PASE
* Scrape/save witness info from PASE

---

Together, the [Anglo-Saxon Charter (ASC)](http://aschart.kcl.ac.uk) and the [Prosopography of Anglo-Saxon England (PASE)](http://pase.ac.uk/) databases contain massive amounts of information on individuals and sources of individuals in England from 600-900 AD. These workbooks will show you how to use Python3 to scrape these two databases and explore and analyze the data in new ways using network and text analysis. This exercise will focus on charters, legal documents which framed power, and the people in them.

We will use the [requests](https://requests.kennethreitz.org/en/master/) and [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) modules to fetch and parse information, [SQLAlchemy](https://www.sqlalchemy.org/) to store it in a local database, [networkx](https://networkx.github.io) to perform network analysis, and finally the [nltk](http://nltk.org) and [cltk](https://cltk.readthedocs.io) modules to do some text analysis on the documents themselves.

![Banner image](../assets/network-banner.png)

*A network of nearly 2,500 Anglo-Saxons, connected by appearing together as witnesses in nearly 500 medieval documents.*

---

Medieval charters were documents of importance and power. They bound institutions and powerful individuals together and created links that were displayed to wider communities. These charters were usually grants of land, often from a king, to a monastery, individual, or some other institution. These documents are not just passive windows into the past, scholars from [Geoffrey Koziol](https://history.berkeley.edu/geoffrey-koziol) to [Phillip Buc](https://ias.ceu.edu/people/philippe-buc) all recognize that the individuals who wrote them intended them to be persuasive and ideological documents. Even the use of minor honoriffic formulae (e.g. *gratia dei*, "By the grace of god") subtlely, but intentionally, linked the different rulers who used them. Historians generally are far more aware of the performative nature of aristocracy. No single title or claim made one an aristocrat so much as constellations of titles and connections with other elites. More importantly, all these claims and connections were constantly advertised and ritually performed (c.f. [Cheyette, Ermengard of Narbonne and the World of the Troubadours](https://www.amazon.com/Ermengard-Narbonne-Troubadours-Conjunctions-Religion/dp/0801489253); [Althoff, Family, Friends and Followers: Political and Social Bonds in Early Medieval Europe](https://www.amazon.com/Family-Friends-Followers-Political-Cambridge/dp/0521779340)). So, traditional historical research suggests these documents were important, and their relationships warrant investigation.

These charters were critical documents in the distribution of land and goods, and all contained witnesses whose authority guaranteed the legitimacy of these documents. These witness lists hold the key to exploring *en masse* what traditional historians have only examined *up close*. You can clearly see the lists of witnesses outlined in the image below.

---

![Image of manuscript outlining witnesses](../assets/charter-large.jpg)

*Charter of King Æthelwulf of Wessex to his minister Æthelmod, 843 A.D.* ([Source](https://tenthmedieval.wordpress.com/2014/08/21/before-you-write-a-charter/))

Spread over hundreds of documents, the totality of their interconnections reveals much about the Anglo-Saxon elite. We mush be careful, however. These documents were heavily ideological, and appearances together in a legal document do not necessarily denote a meaningful connection in practice. But, if we consider these documents another way, we can learn so much more. Looking at the sum of these connections may not tell us about the *reality* of social connections, but it can tell us worlds about the *image* of social relationships **that they wanted to project to others**.

Through the witness lists and the text of the charters themselves, these databases provide an excellent test ground for this approach. They are limited in space and time, allowing us to see change and evolution over time while maintaining a united historical and social context. The datasets are robust, without getting too large to analyze without more powerful tools. Above all, this is a demonstrate of methods and possibilities for other historical investigations.

While appearing together hardly means that there is any real connection between two witnesses, it does mean that they were openly connected and that their social authority was indirectly bound toegether. One goal of these workbooks is to unravel the connections and individuals bound up in these charters. Another goal is also to show you what can be done with an existing data source if you explore it in a new way.

* To technically minded people, I hope to show you how to apply these methods to historical topics.
* To historians, I hope to show you how current historiographical debates directly relate to, and can benefit from, not only the technical possibilities but also the theoretical discussions in various computational methodologies.

Let's get started by importing the various dependencies we need.... run the first Python cell below.

---

## 1) Import Module Dependencies

The cell below loads all other Python packages needed. These are the packages installed when you ran the `pip install -r requirements.txt`. By using the `import` command, as well as the `from * import *` command, we are bringing the objects and functions inside those modules into active memory, allowing us to utilize them below.


You **must** run this before any other cells. Wait until you see `PROCEED` to move on.

In [ ]:
# to help us create filepaths and open files
import os
# so we can quickly read and write CSV files
import csv
# so we can throttle the speed of our web requests
import time
# requests lets us get the raw HTML of web pages in a single line!
import requests
# BeautifulSoup then lets us parse that HTML and get data out of it pythonically
from bs4 import BeautifulSoup
# then we can use sqlalchemy to store all that data 
import sqlalchemy as sql
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
# networkx will let us export the information in a common format (and more in the next module)
import networkx as nx
# IPython.display is just a utility so we can clear the display between each of the hundreds of webpages we scrape
from IPython.display import clear_output

# specify the location of the db file and set a delay on the web scraper for good internet citizenship
DB_PATH = 'sqlite:///charters.db'
SCRAPE_DELAY = 0.5

print('Modules imported! PROCEED')

---

## 2) Create Database/Schema

Before we try to scrape data from the website, we need to have a place to store it. Rather than export the data as a spreadsheet, storing it as a local database will allow us to perform far more powerful kinds of analysis in later steps. In addition, using the database will allow us to easily export our information in a variety of formats.

We are going to use a simple type of database, sqlite. In order to simplify interaction with the database, we are going to use the Python package [SQLAlchemy](https://www.sqlalchemy.org/). This will allow us to easily get related bits of data in a 'Pythonic' way. For example, to get all the people appearing on a charter named `charter` you would write `charter.people`, which will give you a list populated with the relevant items.

The code below first defines the database, and also three 'models'... `Charter`, `Person`, and a third table which will store the relational information (which people appeared on which charters). You can envision the way the data for each these models are stored as something like a spreadsheet. The `Person` model is actually stored in a table named `people`, which has 3 columns.

After defining these models inside Python using [SQLAlchemy](https://www.sqlalchemy.org/), the last line of code actually commits these changes to the database, which should now have three empty tables, named `charters`, `people`, and `charter_witnesses`. If you want to manually examine the database, you can use a free program like [SQLite Browser](http://sqlitebrowser.org/).

I used an entity-relationship modeler to create a diagram of the different tables below, their columns (properties), and which columns link everything together. While charters and people are stored in separate people, it is the third table which allows us to connect them together. This table contains only the IDs of people and the IDs of the charters on which they appear. The diagram below simply visually illustrates what they code in the following cell is going to create in the database.

![EER diagram of our SQLite database](../assets/eer.png)

You will see why I chose the fields I did for each table when you look at the pages for [charters](http://www.aschart.kcl.ac.uk/charters/s0002.html) and for [witnesses](http://www.pase.ac.uk/jsp/ASC/factoid.jsp?factoidKey=24083). There is a lot of information we can easily scrape about the charters... but as for the people, we have little more than a list names/unique IDs. It took a little bit for me to work it out, but for right now, take my choices in columns on faith. You will see how we actually scrape it and store parts of the page in each spot in the following steps.

*To understand more about SQLAlchemy, I highly recommend the following*... [1](https://docs.sqlalchemy.org/en/13/orm/tutorial.html) | [2](https://www.pythoncentral.io/introductory-tutorial-python-sqlalchemy/) | [3](https://auth0.com/blog/sqlalchemy-orm-tutorial-for-python-developers/)

In [ ]:
# create the DB engine
engine = sql.create_engine(DB_PATH, encoding='utf-8')
# create our Base object, each time we want to define a new table, we will simple extend this object
Base = declarative_base()


class Charter(Base):
    """Table for each medieval charter (usually a grant of land from a king to a monastery)."""
    __tablename__ = 'charters'

    id = sql.Column(sql.String, primary_key=True)
    # modern description of the document
    description = sql.Column(sql.String)
    # traditional manuscript identification by sawyer, birch, and kemble
    sawyer = sql.Column(sql.Integer)
    birch = sql.Column(sql.Integer)
    kemble = sql.Column(sql.Integer)
    # british academy number
    british_academy = sql.Column(sql.String)
    # published scholarly source
    source_used = sql.Column(sql.String)
    # current location
    archive = sql.Column(sql.String)
    # language of the document, Latin, Old English, or bilingual
    language = sql.Column(sql.String)
    # a single easy date
    date = sql.Column(sql.Integer)
    # the full text containing more specific date information
    scholarly_date = sql.Column(sql.String)
    # the low and high dates auto-extracted from the scholarly_date string, and their average
    scholarly_date_low = sql.Column(sql.Integer)
    scholarly_date_high = sql.Column(sql.Integer)
    scholarly_date_avg = sql.Column(sql.Float)
    # full text of the charter and any extra notes on the page
    text = sql.Column(sql.Text)
    notes = sql.Column(sql.Text)
    # links to the webpages the charter on asc, pase, and for the charter witnesses on pase
    asc_source = sql.Column(sql.String)
    pase_source = sql.Column(sql.String)
    pase_witnesses = sql.Column(sql.String)
    # n2n relational column containing every witness appearing in the document
    witnesses = sql.orm.relationship('Person', secondary='charter_witnesses', back_populates='charters')
    
    @classmethod
    def scrape(cls, url):
        """This is using the BeautifulSoup function we will define below. See there for more on BeautifulSoup"""
        # get portions of page for following steps
        content = page_soup.body.div.table.find('td', id='content').div
        navbar = content.find('ul', class_='charter-nav')
        # get any navigation links
        pase_witnesses = navbar.find('li', class_='charter-pase-source').find('a')['href']
        # fixes link on asc page, which points to an obsolete address
        pase_witnesses = raw_url.replace('ASC', 'Sources').replace('source.jsp', 'DisplaySource.jsp')
        
    @property
    def record(self):
        """Gets entry data in dictionary format."""
        return {
            'id': self.id,
            'label': self.id,
            'sawyer': self.sawyer,
            'birch': self.birch,
            'kemble': self.kemble,
            'british_academy': self.british_academy,
            'source_used': self.source_used,
            'archive': self.archive,
            'language': self.language,
            'date': self.date,
            'scholarly_date': self.scholarly_date,
            'scholarly_date_low': self.scholarly_date_low,
            'scholarly_date_high': self.scholarly_date_high,
            'scholarly_date_avg': self.scholarly_date_avg,
            'text': self.text,
            'notes': self.notes,
            'asc_source': self.asc_source,
            'pase_source': self.pase_source,
            'pase_witnesses': self.pase_witnesses
        }
    
class Person(Base):
    """Table for each person, usually a witness who appeared in a charter."""
    __tablename__ = 'people'
    
    id = sql.Column(sql.String, primary_key=True)
    # the short byline of information
    description = sql.Column(sql.String)
    # link to the person's webpage on pase
    link = sql.Column(sql.String)
    # n2n relational column containing charters in which the person appears
    charters = sql.orm.relationship('Charter', secondary='charter_witnesses', back_populates='witnesses')
    
    @property
    def record(self):
        """Gets entry data in dictionary format."""
        return {
            'id': self.id,
            'label': self.id + ': ' + self.description,
            'link': self.link
        }
    
    @property
    def earliest_appearance(self):
        """Returns the date of the earliest charter features said person."""
        earliest_charter = None
        for charter in self.charters:
            if not earliest_charter:
                earliest_charter = charter.scholarly_date_avg
            else:
                if charter.scholarly_date_avg < earliest_charter:
                    earliest_charter = charter.scholarly_date_avg
        return earliest_charter

    
    def num_coappearances(self, other_person):
        """Checks the number of times this person appears in the same charters as other_person"""
        total_counter = 0
        for charter in self.charters:
            for other_charter in other_person.charters:
                if charter.id == other_charter.id:
                    total_counter += 1
        return total_counter
    
    
class CharterWitness(Base):
    """Relational n2n table connecting people to the charters in which they were witnesses"""
    __tablename__ = 'charter_witnesses'
    # ids of primary keys of each charter and person
    charter_id = sql.Column(sql.String, sql.ForeignKey('charters.id'), primary_key=True) 
    person_id = sql.Column(sql.String, sql.ForeignKey('people.id'), primary_key=True)
    # specifies if person is a witness, benefactor, grantee, et.c.
    role = sql.Column(sql.String)
    # url to the pase webpage of related people, events, et.c.
    link = sql.Column(sql.String)
    
    @property
    def record(self):
        """Gets entry data in dictionary format."""
        return {
            'charter_id': self.charter_id,
            'person_id': self.person_id,
            'label': self.role,
            'link': self.link
        }


Base.metadata.create_all(engine)

print('Database Configured Successfully! PROCEED')

---

## 3) Scrape ASC and PASE for Charter Info

Our first step will be to get the urls for every charter in the ASC database. Then, using `ASCCharterPage` each page of the ASC database will be requested and parsed into a [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/) object.

As each ASC page is scraped, the link to charter on the PASE database will be used to instantiate a corresponding `PASECharterPage` object. This object will then be used to grab further information about the charter not located on the ASC page. We will also grab the link to people appearing on the charter from each ASC page which we will use in the following step.

*To understand more about requests, I highly recommend the following*... [1](https://www.w3schools.com/python/module_requests.asp) | [2](https://scotch.io/tutorials/getting-started-with-python-requests-get-requests) | [3](https://www.pythonforbeginners.com/requests/using-requests-in-python)

*To understand more about BeautifulSoup, I highly recommend the following*... [1](https://beautiful-soup-4.readthedocs.io/en/latest/) | [2](https://www.pythonforbeginners.com/beautifulsoup/beautifulsoup-4-python) | [3](https://www.datacamp.com/community/tutorials/scraping-reddit-python-scrapy)

---

### 3a) Make Function to Get HTML and Parse into BeautifulSoup

In order to speed up other objects and avoid repeating code, we are going to create a function that will do all the work of getting a url, getting webpage HTML, and then converting it into a BeautifulSoup object. This makes it easy to scrape for information in a Pythonic way. The function gets a URL, if there is an error it calls itself recursively until a retry limit is reached.

We are defining this function here so that the different objects we will define below to scrape different pages can all use it.

In [ ]:
def soup_page(url, tries_left = 5):
    """Receives URL, requests raw html, then returns converted BeautifulSoup object."""
    # declare variable for raw html
    page_html = None
    # ensure tries_left is set and valid, if not set to 5, check if url is valid
    if not tries_left or type(tries_left) != int or tries_left < 0:
        tries_left = 5
    if type(url) != str:
        raise Exception('URL must be a valid string')
    # enforce a time delay between each scrape for good internet citizenship
    time.sleep(SCRAPE_DELAY)
    print('Getting', url)
    # attempt to get page data, decrement tries_left if successful
    try:
        page_html = requests.get(url).text
        tries_left -= 1
    # if an error occured, retry by returning recursively
    except:
        print('Error getting', url)
        if tries_left > 0:
            print('Retrying...')
            return soup_page(url, tries_left=triest_left-1)
        if tries_left <= 0:
            print('Retry limit reached, ABORTING parse of', url)
            return None
    print('Success, souping...')
    # if all went well, return new BeautifulSoup populated with the page html and parser set to html.parser
    return BeautifulSoup(page_html, 'html.parser')

print('Function defined! PROCEED')

---

### 3b) Define Object to Scrape Info from a ASC Charter Page

Now that we have a function which can give us a parser for any page, we can create an object that will parse and quickly give the data for any *one* ASC charter page. We will use this definition to create a bunch of instances to extract the data from the ASC pages. First through, we need to define the object, and which parts of the page should be tied to which properties and methods.

If we look at the [webpage for a single charter](http://www.aschart.kcl.ac.uk/charters/s0002.html), it looks like this (orange border highlighting is mine)...

![Web page for charter S2](../assets/asc-charter.png)

So, we want to extract the text of the charter, but we also want to deal with several other things, indicated by the highlighting. Because we will want to go back later and get information about who was on which inscription, we want to get url of the "List of Witnesses in PASE". This will speed up our scraping later.

You can see that there is also information about its location, dating, and an overall description. We will want to store that. In addition, we will want to devise a way to parse out numerical date information reliably from the text. Since there is some variation on how it appears from charter to charter, we will have to deal with that by defining a method.

In addition, in the text itself are these pesky flags that say things like `INVOCATION`, `PROEM`, or `DISPOSITIVE WORD`. These indicate the different roles that different parts of the text are playing. This could be a very interesting to expand the linguistic angle of this exercise in the future... but for our purposes here, we need to filter them out so that we have the language of the charter in plain text.

So, in order to deal with all this, we are now going to define an object that can handle extracting the data from any given charter page. We won't actually use this object until just a little later, but let's go ahead and define it.

In [ ]:
class ASCCharterPage:
    """Extracts data from a single charter from the ASC database."""
    
    def __init__(self, url):
        """Gets url and pre-parses the html"""
        self._url = url
        # fetch page and parse into beautifulsoup object
        page_soup = soup_page(url)
        # get only portion of page with charter-specific content
        self.data = page_soup.body.div.table.find('td', id='content').div
        # eager load navbar to speed up link retreival
        self._navbar = self.data.find('ul', class_='charter-nav')
            
    @property
    def id(self):
        """Gives the charter ID, with spaces removed."""
        return self.data.div.div.h1.get_text().replace(' ', '')
        
    @property
    def pase_source(self):
        """Url to source in PASE database, if extant, otherwise None."""
        raw_url = self._navbar.find('li', class_='charter-pase-source').find('a')['href']
        # fixes link on asc page, which points to an obsolete address
        return raw_url.replace('ASC', 'Sources').replace('source.jsp', 'DisplaySource.jsp')
        
    @property
    def pase_witnesses(self):
        """URL to list of people appearing on charter in the PASE database."""
        try:
            return self._navbar.find('li', class_='charter-pase-witnesses').find('a')['href']
        # return None if no witnesses are found (i.e. no link exists)
        except:
            return None
    
    @property
    def description(self):
        """Modern description of charter."""
        return self.data.p.get_text()
    
    @property
    def text(self):
        """Full text of the charter, in original language. Editorial clause markings removed."""
        # grab text and convert from latin-1 to utf-8 encoding
        raw_text = self.data.find_all('div')[3].get_text()
        clean_text = bytearray(raw_text, 'latin-1').decode('utf-8')
        # remove text of embedded editorial marks
        remove_phrases = [
            'DATING CLAUSE', 'INVOCATION', 'PROMULGATION PLACE', 'CURSE',
            'DISPOSITIVE WORD', 'BOUNDS', 'PROEM',
        ]
        for remove_phrase in remove_phrases:
            clean_text = clean_text.replace(remove_phrase, '')
        # removes extra whitespace by spliting into list of words and rejoining
        return clean_text
    
print('ASC Charter Scraper Defined! PROCEED')

---

### 3c) Define Object to Scrape Info from a PASE Charter Page

Just as we did above for individual ASC charter pages, we need a different object which can rapidly scrape information from each PASE database page related to a charter. By scraping different information from each database about the same charter, we are able to round out our picture of not only the charter but the people on it.

We we look at a [charter webpage from the PASE database](http://www.pase.ac.uk/jsp/Sources/DisplaySource.jsp?sourceKey=107), we can see it contains more information.

![Page of a charter from PASE](../assets/pase-charter.png)

Again, we are going to define another object which utilizes `soup_page` to target a specific page. This time we can extract more full dating information about each charter... including a simple numerical date, a text description of dating & reasons, as well as a possible date range. This allows us to filter and explore chronological information (when extant) with more nuance.

In [ ]:
class PASECharterPage:
    """Extracts further information about charters from their PASE page."""
    
    def __init__(self, url):
        # fetch page and convert to beautifulsoup object
        self.data = soup_page(url)
        sections = self.data.find_all('div', class_='t01')
        self._charter_info = sections[0].table.tr.td.table.find_all('tr')
        self._source_info = sections[1].table.tr.td.table.find_all('tr')
    
    @property
    def sawyer(self):
        """Returns the sawyer number if extant, otherwise None."""
        try:
            return self._charter_info[0].td.get_text()
        except:
            return None
    
    @property
    def birch(self):
        """Returns the birch number if extant, otherwise None."""
        try:
            return self._charter_info[1].td.get_text()
        except:
            return None
    
    @property
    def kemble(self):
        """Returns the kemble number if extant, otherwise None."""
        try:
            return self._charter_info[2].td.get_text()
        except:
            return None
    
    @property
    def british_academy(self):
        """Returns the British Academy reference, otherwise None."""
        try:
            return self._charter_info[3].td.get_text()
        except:
            return None
    
    @property
    def source_used(self):
        """Gives modern source used."""
        try:
            return self._charter_info[4].td.get_text()
        except:
            return None
    
    @property
    def archive(self):
        """Gives name of modern archive housing the charter."""
        try:
            return self._charter_info[5].td.get_text()
        except:
            return None
    
    @property
    def language(self):
        """Gives language(s) used in charter."""
        try:
            return self._source_info[0].td.get_text()
        except:
            return None
    
    @property
    def date(self):
        """Gives long-form version of date."""
        try:
            return self._source_info[1].td.get_text()
        except:
            return None
    
    @property
    def scholarly_date(self):
        """Gives short-form version of date."""
        try:
            return self._source_info[2].td.get_text()
            
        except:
            return None
        
    @property
    def scholarly_date_low(self):
        """Will return low date if date range exists, otherwise return scholarly_date"""
        try:
            return int(self.scholarly_date.split('x')[0].replace(' ', ''))
        except:
            return int(self.scholarly_date)
        
    @property
    def scholarly_date_high(self):
        """Will return high date if date range exists, otherwise return scholarly_date"""
        try:
            return int(self.scholarly_date.split('x')[1].replace(' ', ''))
        except:
            return int(self.scholarly_date)
        
    @property
    def scholarly_date_avg(self):
        """Returns mean date if date range exists, otherwise return scholarly_date"""
        low_date = self.scholarly_date_low
        high_date = self.scholarly_date_high
        if low_date == high_date:
            return low_date
        else:
            return round((low_date + high_date) / 2, 2)
    
    @property
    def notes(self):
        """Gives miscellaneous notes on charter."""
        try:
            return self.data.find('div', class_='rec').p.get_text()
        except:
            return None


print('PASE Charter Scraper Defined! PROCEED')

---

### 3d) Scrape Charter Info from ASC and PASE

Now that we have both scrapers defined, we can put them to use! Time to download the information and store it in our database. We will use the two scraper objects we created in steps 3b & 3c to get information off of the website and then store it our local database using the models we defined in step 2.

To do this, we must first go to the main "browse" page of the database, scrape it to get the URL of each charter, and then traverse the list. For the ASC database, the best page to do this is the [Index by Sawyer](http://www.aschart.kcl.ac.uk/idc/idx_sawyerNo.html). Which you can see below.

![ASC Index by Sawyer webpage](../assets/asc-by-sawyer.png)

*This is where it finally goes into practice!*

1. We want to use our `soup_page` function to fetch & parse the main page.
2. We want to extract the links from each of the sawyers, highlighted in orange above
3. We want to create ASC scraper objects with each of the links and extract data from each page
4. We want to create PASE scraper objects with the urls from each ASC page, and extract the PASE data
5. Store all the information we scrape with both into the database we created above

In [ ]:
# first, we need to open a session with the local database
session = sessionmaker(bind=engine)()

# before scraping information, we need to get the urls for every ASC charter
asc_links = []
charter_soup = None
# fetch page and parse into beautifulsoup object
page_soup = soup_page('http://www.aschart.kcl.ac.uk/idc/idx_sawyerNo.html')
# get only portion of page with charter-specific content
charter_soup = page_soup.body.div.table.find('tr', class_='r02').find('td', id='content').div
# looping through each section and group of rulers
for ruler_section in charter_soup.find_all('ul', class_='asc-expand'):
    for ruler_group in ruler_section.find_all('li'):
        # get relative links from <a> tags append full link to self.data by adding root_url
        for charter_link_wrapper in ruler_group.find_all('li'):
            # when joining, trim initial two charcters (which are unwanted dots '.')
            asc_links.append('http://www.aschart.kcl.ac.uk' + charter_link_wrapper.a['href'][2:])

charter_counter = 0
# loop through each charter found online
for charter_url in asc_links:
    clear_output()
    print('Gathering charters: {} of {}...'.format(charter_counter + 1, 467))
    try:
        asc_charter_page = ASCCharterPage(charter_url)
        pase_charter_page = PASECharterPage(asc_charter_page.pase_source)
        # make new charter and add to session
        session.add(Charter(
            id=asc_charter_page.id,
            description=asc_charter_page.description,
            sawyer=pase_charter_page.sawyer,
            birch=pase_charter_page.birch,
            kemble=pase_charter_page.kemble,
            british_academy=pase_charter_page.british_academy,
            source_used=pase_charter_page.source_used,
            archive=pase_charter_page.archive,
            language=pase_charter_page.language,
            date=pase_charter_page.date,
            scholarly_date=pase_charter_page.scholarly_date,
            scholarly_date_low=pase_charter_page.scholarly_date_low,
            scholarly_date_high=pase_charter_page.scholarly_date_high,
            scholarly_date_avg=pase_charter_page.scholarly_date_avg,
            text=asc_charter_page.text,
            notes=pase_charter_page.notes,
            asc_source=asc_charter_page._url,
            pase_source=asc_charter_page.pase_source,
            pase_witnesses=asc_charter_page.pase_witnesses
        ))
    except Exception as e:
        print('Error loading page at', charter_url, '(skipped)')
    charter_counter += 1
# commit all changes to the local db
session.commit()
session.close()

print('Charters successfully scraped! PROCEED')

---

## 4) Scrape PASE for People in Charters

Great, so we have lots of information about the charters, but knowing something about the social context would add a lot to this. So, time to define yet a third scraper. This one is going to go through the PASE database, but this time it is going to seek information about which people appeared in which medieval charters. This particular exercise doesn't scrape much metadata about the people. Nevertheless, each person has a unique ID number. This is going to allow us to connect people together in a network based upon their shared appearances at witnesses.

Looking at a [webpage from PASE](http://www.pase.ac.uk/jsp/ASC/factoid.jsp?factoidKey=24083) containing all related people, events, et.c., we can see that the witnesses are clearly listed by their name/unique ID.

![Page from PASE containing charter witnesses](../assets/pase-witnesses.png)

We are simply going to extract the name/unique ID of each person and the short byline about them, so that will not be difficult. Just like before, we need to define an object to quickly extract the data.

---

### 4a) Define Object to Scrape Info from a PASE Charter Witnesses Page

We need to define an object to scrape the page. This one represents a single page from the PASE database containing the list of witnesses that appear on a single charter. There are only as many pages are there are charters, and since we are not going further with metadata about each person, that means we thankfully do not have to extract thousands of individual pages.

In [ ]:
class PASEWitnesses:
    """Gets basic information about people on the charter from a PASE witnesses page"""
    
    def __init__(self, url):
        self._url = url
        self._soup = soup_page(self._url)
        # get only portion of page with relevant data
        try:
            self.data = self._soup.find('div', class_='rec').find('ul').find_all('li')
        # sometimes notes preceed data, in which case get second div with class rec
        except:
            self.data = self._soup.find_all('div', class_='rec')[1].find('ul').find_all('li')
    @property
    def witnesses(self):
        witness_list = []
        for witness_entry in self.data:
            witness_link_element = witness_entry.find('a')
            try:
                witness_role = witness_entry.find('strong').get_text()
            except:
                witness_role = 'Witness'
            witness_list.append({
                    'role': witness_role,
                    'name': witness_link_element.get_text().lstrip(),
                    'link': witness_link_element['href'].replace('../', 'http://www.pase.ac.uk/jsp/'),
                    'description': witness_entry.find('em').get_text()
                })
            # look for nested witnesses, sometimes buried in recursive em tags
            nested_witnesses_element = None
            try:
                nested_witnesses_element = self._soup.find('div', class_='rec').find('ul').find('em')
            except:
                nested_witnesses_element = self._soup.find_all('div', class_='rec')[1].find('ul').find('em')
            while nested_witnesses_element is not None:
                nested_witnesses = nested_witnesses_element.find_all('li')
                for nested_witness_entry in nested_witnesses:
                    nested_witness_link_element = witness_entry.find('a')
                    try:
                        nested_witness_role = nested_witness_entry.find('strong').get_text()
                    except:
                        nested_witness_role = 'Witness'
                    witness_list.append({
                        'role': nested_witness_role,
                        'name': nested_witness_link_element.get_text(),
                        'link': nested_witness_link_element['href'].replace('../', 'http://www.pase.ac.uk/jsp/'),
                        'description': nested_witness_entry.find('em').get_text()
                    })
                nested_witnesses_element = nested_witnesses_element.find('em')
        return witness_list
    

print('PASE Witnesses Scraper Defined! PROCEED')

### 4b) Scrape Witness Info

As before, we will instantiate our scraper, creating one for each charter. Traversing the list, we will store the information in the Witness and CharterWitness tables. This will allow us to get network information.

In [ ]:
# first, we need to open a session with the local database
Session = sessionmaker(bind=engine)
session = Session(autoflush=False)

charter_counter = 0
# now we want to query our local db for every charter, which is returned as a Charter object
for charter in session.query(Charter):
    clear_output()
    print('Witnesses found in {} charters (so far) of {}...'.format(charter_counter + 1, 467))
    # then get the link to the corresponding PASE page with witness information, skip if there is no link
    witnesses_link = charter.pase_witnesses
    if witnesses_link is not None:
        # use the scraper to get witness data as list of dicts, only proceed if results were found
        witness_list = PASEWitnesses(witnesses_link).witnesses
        # loop through the list of witnesses
        for witness in witness_list:
            # query to see if person already exists in db, if no results found, then add them
            person_query = session.query(Person).filter(Person.id == witness['name'])
            # if any results are in the list, it will set person_found to True
            person_found = False
            for person in person_query:
                person_found = True
            if not person_found:
                try:
                    session.add(Person(
                        id=witness['name'],
                        description=witness['description'],
                        link=witness['link']
                    ))
                    print('Added person {}'.format(witness['name']))
                    session.commit()
                except:
                    session.rollback()
            # add charter/person relationship information to `charter_witnesses` table created above
            try:
                session.add(CharterWitness(
                    charter_id=charter.id,
                    person_id=witness['name'],
                    role=witness['role'],
                    link=str(witnesses_link)
                ))
                session.commit()
            except:
                session.rollback()
            print('Added person/charter relationship {} -> {}'.format(witness['name'], charter.id))
    charter_counter += 1
    
# commit all changes to the local db
session.close()

print('Witnesses successfully scraped! PROCEED')

## 5) Preview Results

Let's see if it worked!

In [ ]:
# first, we need to open a session with the local database
Session = sessionmaker(bind=engine)
session = Session(autoflush=False)

charters = session.query(Charter)

charter_counter = 0
for charter in charters:
    charter_counter += 1
    
print('{} total charters scraped.'.format(charter_counter))
print('Preview of first charter...')
print(charters[0].text.replace('\n', ''))

print('Preview successfull! PROCEED')

## 6) Export Data

This portion is optional, but allows you to immediately export your results for other platforms. All data will be saved to the `/data` directory. If you are using this on Binder, you can navigate there and download the data to your local device.

### 6a) Charters as Plain Text Files

Some text analysis programs like to use plain .txt files. This cell will generate a folder full of .txt files, one for each charter. Works well with [Voyant](https://voyant-tools.org/).

In [ ]:
print('Exporting charters as a folder of .txt files')

session = sessionmaker(bind=engine)()
charters = session.query(Charter)
session.close()

export_folder_path = os.path.join('../data', 'raw_charter_txts')
# ensure parent directory exists, if not, make it
if not os.path.exists(export_folder_path):
    os.makedirs(export_folder_path)

for charter in charters:
    output_path = os.path.join(export_folder_path, charter.id + '.txt')
    with open(output_path, mode='w+') as txtfile:
        txtfile.write(charter.text)
        
print('Successfully exported to data/raw_charter_txts/! PROCEED')

### 6b) Charters as CSV Files

A more standard format are CSV files. This cell generates one .csv file, with a column for every metadata field. This works well for the majority of text analysis programs and the ability to connect metadata expands the possible levels of analysis. Works well with programs like [Overview](https://www.overviewdocs.com/) or [Orange3](https://orange.biolab.si)

In [ ]:
print('Exporting charters as a .csv file')

session = sessionmaker(bind=engine)()
charters = session.query(Charter)
session.close()

export_path = os.path.join('../data', 'raw_charters.csv')
# ensure parent directory exists, if not, make it
if not os.path.exists(export_folder_path):
    os.makedirs(export_folder_path)

fieldnames = [
    'id', 'description', 'sawyer', 'birch', 'kemble', 'british_academy', 'source_used', 'archive',
    'language', 'date', 'scholarly_date', 'scholarly_date_low', 'scholarly_date_high',
    'scholarly_date_avg', 'text', 'notes', 'asc_source', 'pase_source', 'pase_witnesses'
]

# open the csv file for writing
with open(export_path, mode='w+', encoding='utf-8') as csvfile:
    # create the csv writer for the file and write the header row
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    # loop through charters
    for charter in charters:
        new_record = charter.record
        # move the 'label' property to the 'description' property
        new_record['description'] = new_record['label']
        del(new_record['label'])
        # write new csv row
        writer.writerow(new_record)
        
print('Successfully exported to data/raw_charters.csv! PROCEED')

### 6c) Coappearances of Witnesses as Network File

Now that we have information about charters, and witnesses, and their relationships, we can use the networkx package to easily create a .gexf file, the most commonly used network graph format. It works very well with [Gephi](https://gephi.org/). 

**This step is CRITICAL if you want to do the next workbook!**

In [ ]:
session = sessionmaker(bind=engine)()
witnesses = session.query(Person)
session.close()

print('Building network graph...', end='')
# create an empty networkx graph
witness_network = nx.Graph(
    label='',
    link=None,
    weight=1,
    type='Undirected',
    community=None,
    degree=0,
    degree_centrality=0,
    betweeness_centrality=0,
    eigenvector_centrality=0,
    closeness_centrality=0,
    harmonic_centrality=0,
)

# populate the nodes with each witness record
for person in witnesses:
    witness_network.add_node(person.id, **person.record)

counter = 0
# to build edges, we need to compare every witness against each other for the number of times they appear
for index, person in enumerate(witnesses):
    for other_person in witnesses[index + 1:]:
        counter += 1
        if counter % 50000 == 0:
            print('.', end='')
        num_coappearances = person.num_coappearances(other_person)
        # only add an edge if they actually appeared together
        if num_coappearances > 0:
            witness_network.add_edge(
                person.id,
                other_person.id,
                label='{} -> {}'.format(person.id, other_person.id),
                weight=person.num_coappearances(other_person),
                type='Undirected'
            )

print(' Done!\n\nExporting network to `data/witness_network.gexf`...')
nx.write_gexf(witness_network, '../data/witness_network.gexf')
print('Exported network successfully! PROCEED TO NEXT WORKBOOK')

---
#### Proceed to the next workbook!
---